In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import utils
from tensorflow.python.client import device_lib
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Cropping2D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta, Nadam
from matplotlib import pyplot as plt
%matplotlib inline
device_lib.list_local_devices()

WIDTH, HEIGHT = 240, 240
SOURCE_FILE = os.path.join('..', 'data', 'week3', 'week3_just_tumor_{}.csv')
SOURCE_FILES = [SOURCE_FILE.format(i) for i in range(25)]

In [16]:
hgg = 0
lgg = 0
for i, file_ in enumerate(SOURCE_FILES):
    print('Processing chunk file #{}'.format(i))
    # chunksize=100 -> returns TextFileReader for iteration
    # print('from', i*999, 'to', i*999 + 998)
    labels = pd.read_csv(file_, dtype=np.uint8, nrows=999, usecols=['label'], squeeze=True)
    hgg += labels.sum()
    lgg += 999 - labels.sum()
    print('Update: ', hgg, ' ', lgg)

Processing chunk file #0
('Update: ', 0, ' ', 999)
Processing chunk file #1
('Update: ', 0, ' ', 1998)
Processing chunk file #2
('Update: ', 0, ' ', 2997)
Processing chunk file #3
('Update: ', 0, ' ', 3996)
Processing chunk file #4
('Update: ', 0, ' ', 4995)
Processing chunk file #5
('Update: ', 0, ' ', 5994)
Processing chunk file #6
('Update: ', 0, ' ', 6993)
Processing chunk file #7
('Update: ', 0, ' ', 7992)
Processing chunk file #8
('Update: ', 0, ' ', 8991)
Processing chunk file #9
('Update: ', 0, ' ', 9990)
Processing chunk file #10
('Update: ', 0, ' ', 10989)
Processing chunk file #11
('Update: ', 0, ' ', 11988)
Processing chunk file #12
('Update: ', 253, ' ', 12734)
Processing chunk file #13
('Update: ', 1252, ' ', 12734)
Processing chunk file #14
('Update: ', 2251, ' ', 12734)
Processing chunk file #15
('Update: ', 3250, ' ', 12734)
Processing chunk file #16
('Update: ', 4249, ' ', 12734)
Processing chunk file #17
('Update: ', 5248, ' ', 12734)
Processing chunk file #18
('Upda

In [7]:
# columns = list(range(WIDTH * HEIGHT)) + ['label']
# index = list(range(999 * 10))
chunks = []
for i, file_ in enumerate(SOURCE_FILES[:0] + SOURCE_FILES[-1:]):
    print('Processing chunk file #{}'.format(i))
    # chunksize=100 -> returns TextFileReader for iteration
    print('from', i*999, 'to', i*999 + 998)
    chunk_df = pd.read_csv(file_, dtype=np.uint8, nrows=999)
    chunks.append(chunk_df)
df = pd.concat(chunks).dropna(how='all')

Processing chunk file #0
('from', 0, 'to', 998)


### Preprocessing

In [8]:
def preprocess_dataset(dataset):
    # Pop labels and transform them to vectors
    y = dataset.pop('label')
    y = y.values.reshape((-1, 1))
    # Reshape the features for CNN
    # X = dataset.as_matrix().reshape(dataset.shape[0], 1, WIDTH, HEIGHT).astype(np.float32)
    X = dataset.as_matrix().reshape(dataset.shape[0], 1, WIDTH, HEIGHT).astype(np.float32)
    # Norm datax
    X /= 255
    # Convert labels to categorical values
    y = keras.utils.to_categorical(y, 2)
    return X, y

def get_shuffled_splitted_data():
    # Shuffle and split data into: 70% train, 20% test, 10% validation
    train, test, val = np.split(data.sample(frac=1), [int(.7*len(data)), int(.85*len(data))])    
    # Extract labels, normalize, preprocess for keras
    X_train, y_train = preprocess_dataset(train)
    X_test, y_test = preprocess_dataset(test)
    X_val, y_val = preprocess_dataset(val)
    return X_train, y_train, X_test, y_test, X_val, y_val

# Collect new dataset containing sagittal images including scull and tumor
data = df
X_train, y_train, X_test, y_test, X_val, y_val = get_shuffled_splitted_data()

### Training the model (CNN)

In [9]:
def setup_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(1, WIDTH, HEIGHT)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Nadam(),
                  metrics=['accuracy'])
    return model

def setup_simple_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(7, 7), input_shape=(1, WIDTH, HEIGHT)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, kernel_size=(5, 5), input_shape=(1, WIDTH, HEIGHT)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(7, 7)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Nadam(),
                  metrics=['accuracy'])
    return model

with tf.device('/gpu:0'):
    K.set_image_dim_ordering('th')
    batch_size = 36
    num_classes = 2
    epochs = 10
    # For storing the validation loss values
    history = keras.callbacks.History()
    # Train model
    model = setup_simple_model()
    model_results = model.fit(X_train, y_train,
                              batch_size=batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(X_test, y_test),
                              callbacks=[history])
    # Evaluate model on validation set
    print('\nValidate model on {} unknown validation samples:'.format(X_val.shape[0]))
    val_score = model.evaluate(X_val, y_val, verbose=0)
    print('Val loss:', val_score[0])
    print('Val accuracy:', val_score[1])

Train on 699 samples, validate on 150 samples
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# Store output in output.py
%save -a output _

In [ ]:
'Hi'

In [ ]:
# Plot validation loss
utils.plot_history(model_results)
plt.savefig('model_history.png')

In [ ]:
plt.figure()

### Plotting results

In [ ]:
import imp
imp.reload(utils)

In [ ]:
# Always same results
np.random.seed(1)
y_val_pred = model.predict(X_val, batch_size=32, verbose=0)
y_val_pred = np.round(y_val_pred).astype(int)
is_lgg = y_val.argmax(axis=1) == 0
utils.plot_predicted_samples(4, X_val[is_lgg], y_val[is_lgg], y_val_pred[is_lgg], 'Validation set - LGG', (WIDTH, HEIGHT))
utils.plot_predicted_samples(4, X_val[is_lgg == False], y_val[is_lgg == False], y_val_pred[is_lgg == False], 'Validation set - HGG', (WIDTH, HEIGHT))

In [ ]:
# Always same results (grayscale)
np.random.seed(1)
utils.plot_predicted_samples(4, X_val[is_lgg], y_val[is_lgg], y_val_pred[is_lgg], 'Validation set - LGG', (WIDTH, HEIGHT))
utils.plot_predicted_samples(4, X_val[is_lgg == False], y_val[is_lgg == False], y_val_pred[is_lgg == False], 'Validation set - HGG', (WIDTH, HEIGHT))

### Analysis

Our network converges unexpectedly quickly. This might be due to the following problems:
- The tumor images are resized and are somewhat blurred
- For some tumor images you can easily see that it has been created by two seperate pictures
- The healthy images contain sometimes a very different shape (e.g. above #108)

### Model structure 

In [ ]:
# Reload utils model if you changed a function
# import imp
# imp.reload(utils)
utils.plot_model(model, 'skull_classification_model.png')

In [ ]:
model.summary()